## **Purpose**
 In this notebook we will cleaning our VGChartz/MetaCritic dataset.
* We will:
    * load the data
    * Drop Duplicate
    * Drop the total_shipped column
    * Drop global_sales values that equal 0
    * Fix the multiplayer column
    * Fill in missing developer values
    * Fill in missing critic_count values
    * Fix meta_esrb column
    * save the cleaned dataset as a dataframe in a specified folder

## **Datasets**
200.csv


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os

In [2]:
if not os.path.exists("../data/prep/200.csv"):
    print("Missing dataset file")
else:
    print("Success!")

Success!


In [3]:
df = pd.read_csv("../data/prep/200.csv")
df.sample()

,name,total_shipped,developer,rank,platform,release_date,publisher,na_sales,eu_sales,jp_sales,...,meta_developer,meta_critic_score,meta_critic_count,meta_user_score,meta_user_count,meta_esrb,meta_genre,meta_multiplayer,meta_full_url,release_year
3269,Bust A Groove 2,N/A,Metro,681,PS,1999-03-31,Enix,40000,30000,180000,...,Metro,66,8.0,8.4,40,E,Miscellaneous,yes,https://www.metacritic.com/game/playstation/bu...,1999


## **Drop Duplicates**

As we know for certain that all meta_full_url's should be unique, any rows with the same url need to be handled.
We can easily view duplicated rows using the DataFrame.duplicated() method of pandas. We can pass keep=False to view all duplicated rows, or leave the defaults (keep='first') to skip over the first row and just show the rest of the duplicates. We can also pass a column label into subset if we want to filter by a single column. As we only want to remove the extra rows, we can keep the default behaviour.

In [4]:
duplicate_rows = df[df.duplicated(subset='meta_full_url')]

print('Duplicate rows to remove:', duplicate_rows.shape[0])

duplicate_rows.head()

Duplicate rows to remove: 99


,name,total_shipped,developer,rank,platform,release_date,publisher,na_sales,eu_sales,jp_sales,...,meta_developer,meta_critic_score,meta_critic_count,meta_user_score,meta_user_count,meta_esrb,meta_genre,meta_multiplayer,meta_full_url,release_year
184,Metal Gear Solid HD Collection,N/A,Bluepoint Games,321,PS3,2011-11-08,Konami Digital Entertainment,520000,440000,0,...,"Genki, Bluepoint Games",89,50.0,8.9,471,M,Miscellaneous,yes,https://www.metacritic.com/game/playstation-3/...,2011
185,Metal Gear Solid HD Collection,N/A,Bluepoint Games,659,PS3,2011-11-08,Konami,320000,130000,0,...,"Genki, Bluepoint Games",89,50.0,8.9,471,M,Miscellaneous,yes,https://www.metacritic.com/game/playstation-3/...,2011
186,Metal Gear Solid HD Collection,N/A,Bluepoint Games,659,PS3,2011-11-08,Konami,320000,130000,0,...,"Genki, Bluepoint Games",89,50.0,8.9,471,M,Miscellaneous,yes,https://www.metacritic.com/game/playstation-3/...,2011
234,God of War III Remastered,N/A,SCEA Santa Monica Studio,395,PS4,2015-07-14,Sony Computer Entertainment,400000,330000,20000,...,Wholesale Algorithms,81,67.0,8.1,919,M,Action Adventure,no,https://www.metacritic.com/game/playstation-4/...,2015
235,God of War III Remastered,N/A,SCEA Santa Monica Studio,2683,PS4,2015-07-15,Sony Computer Entertainment Europe,0,20000,0,...,Wholesale Algorithms,81,67.0,8.1,919,M,Action Adventure,no,https://www.metacritic.com/game/playstation-4/...,2015


In [5]:
df = df.drop_duplicates(subset='meta_full_url')

Next I want to see how many values there are missing in each column. I am using a function I found in one of William Koehrsen blogs on Medium.

In [6]:
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : "Missing Values", 1 : "% of Total Values"})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        "% of Total Values", ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [7]:
missing_values_table(df)

Your selected dataframe has 26 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
meta_multiplayer,2099,26.0
meta_critic_count,839,10.4
meta_esrb,80,1.0
meta_developer,13,0.2


## **Drop total_shipped column**

The total_shipped column seems to have a lot of missing values but isnt coming up on our missing values table because its null values are stored as 'N/A' strings instead of np.nan values.

In [8]:
df['total_shipped'] = df['total_shipped'].replace('N/A', np.nan, regex=True)

In [9]:
missing_values_table(df)

Your selected dataframe has 26 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
total_shipped,7719,95.6
meta_multiplayer,2099,26.0
meta_critic_count,839,10.4
meta_esrb,80,1.0
meta_developer,13,0.2


Now we can see how much of the total_shipped column is missing values.It has over 95% of its values missing so I dont think that this column is useful to us. We will just drop the whole column.

In [10]:
df=df.drop(['total_shipped'], axis = 1)

## **Drop missing global sales**

In [11]:
df[df['global_sales']==0].head()

,name,developer,rank,platform,release_date,publisher,na_sales,eu_sales,jp_sales,other_sales,...,meta_developer,meta_critic_score,meta_critic_count,meta_user_score,meta_user_count,meta_esrb,meta_genre,meta_multiplayer,meta_full_url,release_year
1624,Garshasp: The Monster Slayer,Dead Mage Studio,3017,PC,2011-05-09,Unknown,0,0,0,0,...,Dead Mage Studio,49,19.0,6.2,55,NaN,Action Adventure,no,https://www.metacritic.com/game/pc/garshasp-th...,2011
1625,Metal Gear Online,Kojima Productions,3018,PS3,2008-06-12,Konami Digital Entertainment,0,0,0,0,...,Kojima Productions,81,10.0,7.7,69,M,Action Adventure,NaN,https://www.metacritic.com/game/playstation-3/...,2008
1626,Of Orcs and Men,Cyanide Studio,3020,PS3,2012-10-11,Focus Home Interactive,0,0,0,0,...,"Cyanide, Spiders",57,8.0,6.3,43,M,Role-Playing,NaN,https://www.metacritic.com/game/playstation-3/...,2012
1627,Big Bumpin',Blitz Games,3022,X360,2006-11-19,King Games,0,0,0,0,...,Blitz Games,63,16.0,5.8,13,E,Driving,yes,https://www.metacritic.com/game/xbox-360/big-b...,2006
1628,Resident Evil 5: Gold Edition,Capcom,3023,X360,2010-03-09,Capcom,0,0,0,0,...,Capcom,63,NaN,8.9,29,M,Miscellaneous,NaN,https://www.metacritic.com/game/xbox-360/resid...,2010


In [12]:
df['global_sales'] = df['global_sales'].replace(0, np.nan)

In [13]:
df=df.dropna(subset = ['global_sales'])

## **Fix meta_multiplayer column**
When scraping multiplayer data from the metacritic website over 25% of the games had no information on whether it was a multiplayer or single player game. I think its fair to assume that all these games are single player as the website would highlight if it was a multiplayer.

In [14]:
df['meta_multiplayer'] = df['meta_multiplayer'].replace(np.nan, 'no')

## **Fix meta_developer column**
When I was collecting the data I decided to collect the developer name from both the VGChartz and MetaCritic website just incase one of the websites was missing values and I could replace them missing values with the developer name from the other website. It looks like the the developer column is missing no values however it contains 'unknown' values.

In [15]:
df['developer'] = df['developer'].replace('Unknown', np.nan, regex=True)

In [16]:
missing_values_table(df)

Your selected dataframe has 25 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
meta_critic_count,767,10.0
developer,97,1.3
meta_esrb,57,0.7
meta_developer,9,0.1


As you can see there is a few missing developer names. I will replace the missing VGChatrz developer name with the metacritic developer name. If neither have the developer name then I will just drop the row

In [17]:
df['developer'] = df['developer'].replace(np.nan, df['meta_developer'], regex=True)

In [18]:
df['developer'] = df['developer'].dropna()

Now I will just drop the meta_developer column as there is no need for it.

In [19]:
df=df.drop(['meta_developer'], axis = 1)

## **Fill in missing meta_critic_count values**

In [20]:
df2=df.groupby(['developer','game_genre'])['meta_critic_count'].mean().reset_index() 
df2['meta_critic_count']=df2['meta_critic_count'].round()
df2.columns=['developer','game_genre','new_meta_critic_count']
df2.sample(4)

,developer,game_genre,new_meta_critic_count
453,Capcom,Racing,18.0
2169,Square Enix,Strategy,39.0
634,Data Design Interactive,Racing,NaN
1282,Koei/Inis,Sports,12.0


In [21]:
df=df.merge(df2,left_on=['developer','game_genre'], right_on=['developer','game_genre'])
df.sample(4)

,name,developer,rank,platform,release_date,publisher,na_sales,eu_sales,jp_sales,other_sales,...,meta_critic_score,meta_critic_count,meta_user_score,meta_user_count,meta_esrb,meta_genre,meta_multiplayer,meta_full_url,release_year,new_meta_critic_count
7533,Lock's Quest,5TH Cell,503,DS,2008-09-08,THQ,80000,0,0,10000,...,80,32.0,8.3,33,E,Strategy,yes,https://www.metacritic.com/game/ds/locks-quest,2008,32.0
5046,Naruto: Path of the Ninja 2,TOSE,819,DS,2008-10-15,D3 Publisher,220000,0,0,20000,...,62,18.0,7.4,7,E10+,Role-Playing,yes,https://www.metacritic.com/game/ds/naruto-path...,2008,17.0
6533,NHL 14,EA Canada,676,X360,2013-09-10,EA Sports,400000,90000,0,50000,...,81,30.0,6.0,95,E10+,Sports,no,https://www.metacritic.com/game/xbox-360/nhl-14,2013,25.0
5231,Dokapon Kingdom,Sting,1131,Wii,2008-10-14,Atlus,120000,0,0,10000,...,73,11.0,8.1,20,E10+,Role-Playing,no,https://www.metacritic.com/game/wii/dokapon-ki...,2008,16.0


Lets check and see if our expected meta_critic_count is a good estimate

In [22]:
diff_df=pd.DataFrame()
diff_df['name'] =  df['name']
diff_df['Score_diff'] = df['meta_critic_count'] - df['new_meta_critic_count'] 
diff_df['Score_diff']= abs(diff_df['Score_diff'])
print('The highest difference is:' , diff_df['Score_diff'].max())
print('The average difference is:', diff_df['Score_diff'].mean())

The highest difference is: 69.0
The average difference is: 8.359197907585004


In [23]:
(len(diff_df[diff_df['Score_diff'] == 0])/len(diff_df) )* 100

20.669368544907833

We have correctly predicted the critic count Over 20% of the time.

In [24]:
(len(diff_df[diff_df['Score_diff']> 20])/len(diff_df) )* 100

9.60909922865734

Only about 9% of the dataset got a predicted score of over 20 people wrong. This means most of the datasets predicted score is quite close to its actually one. I am quite happy with these stats and I feel that using this mean score to will in missing values is pretty accurate.

In [25]:
df['meta_critic_count'] = df['meta_critic_count'].replace(np.nan, df['new_meta_critic_count'], regex=True)
df=df.drop('new_meta_critic_count',axis=1)

## **Fix meta_esrb values**

In [26]:
df.meta_esrb.unique()

array(['M', 'AO', 'T', 'E10+', 'E', nan, 'EC', 'K-A', 'RP'], dtype=object)

There are only 5 main esrb rating - M,T,E10+,EC and E, however we have 7. K-A means its suitable for kids to adults, this means it can be changed to E which stands for everybody. AO stands for adults only, this means it can be changed to M which stands for mature. Finally Titles listed as RP (Rating Pending) have not yet been assigned a final ESRB rating so i will just leave this as RP.

In [27]:
df['meta_esrb'] = df['meta_esrb'].replace('AO', 'M')
df['meta_esrb'] = df['meta_esrb'].replace('K-A', 'E')

In [28]:
df.groupby('meta_esrb').count()['name']

meta_esrb
E       2295
E10+    1196
EC         2
M       1558
RP         1
T       2540
Name: name, dtype: int64

In [29]:
df[df['meta_esrb']=='RP']

,name,developer,rank,platform,release_date,publisher,na_sales,eu_sales,jp_sales,other_sales,...,meta_game_name,meta_critic_score,meta_critic_count,meta_user_score,meta_user_count,meta_esrb,meta_genre,meta_multiplayer,meta_full_url,release_year
7530,Supreme Ruler: Cold War,Paradox Interactive,690,PC,2011-07-19,Paradox Interactive,0,30000,0,0,...,Supreme Ruler: Cold War,63,12.0,6.7,28,RP,Strategy,yes,https://www.metacritic.com/game/pc/supreme-rul...,2011


Because there was only one game with a RP rating I decided to look that game up. It has actually been given a rating og E10+ according to this website: https://www.esrb.org/ratings/31253/Supreme+Ruler+Cold+War/. I will fill in this value for the game 'Supreme Ruler: Cold War' meta_esrb.

In [30]:
df['meta_esrb'] = df['meta_esrb'].replace('RP', 'E10+', regex=True)

## **Drop remaining empty values**

In [31]:
df = df.dropna()

In [32]:
missing_values_table(df)

Your selected dataframe has 24 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


## **Save to csv file**

In [33]:
df=df.set_index('name')
df.to_csv('../data/prep/300.csv')